# Aspect-Based Sentiment Analysis Exploration

This notebook demonstrates the exploration and testing of different ABSA implementations.

## Overview
- **Lexicon-based ABSA**: Using spaCy and sentiment lexicons
- **Transformer-based ABSA**: Using pre-trained Hugging Face models
- **LLM-based ABSA**: Using local LLM through Ollama

## Setup and Imports


In [ ]:
import sys
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from base import AspectSentiment
from lexicon_absa import LexiconABSA
from transformer_absa import TransformerABSA
from llm_absa import LLMABSA
from utils import (
    load_test_data, save_results, calculate_accuracy, 
    calculate_precision_recall_f1, benchmark_analyzer,
    create_sample_data, print_analysis_results, ensure_data_directory
)

print("Imports successful!")


## Initialize Analyzers

Let's initialize all three ABSA implementations:


In [ ]:
analyzers = {}
results = {}

print("Initializing analyzers...")

try:
    lexicon_analyzer = LexiconABSA()
    analyzers['Lexicon'] = lexicon_analyzer
    print("✓ LexiconABSA initialized successfully")
except Exception as e:
    print(f"✗ LexiconABSA failed to initialize: {e}")

try:
    transformer_analyzer = TransformerABSA()
    analyzers['Transformer'] = transformer_analyzer
    print("✓ TransformerABSA initialized successfully")
except Exception as e:
    print(f"✗ TransformerABSA failed to initialize: {e}")

try:
    llm_analyzer = LLMABSA()
    analyzers['LLM'] = llm_analyzer
    print("✓ LLMABSA initialized successfully")
except Exception as e:
    print(f"✗ LLMABSA failed to initialize: {e}")

print(f"\nSuccessfully initialized {len(analyzers)} analyzers")


## Load Test Data

Let's load our test data and create sample data if needed:


In [ ]:
data_file = ensure_data_directory()
test_data = load_test_data(data_file)

if not test_data:
    print("Creating sample test data...")
    test_data = create_sample_data()
    save_results(test_data, data_file)

print(f"Loaded {len(test_data)} test samples")
print("\nSample test data:")
for i, item in enumerate(test_data[:3]):
    print(f"{i+1}. {item['text']}")
    print(f"   Expected: {[f\"{a['aspect']} ({a['sentiment']})\" for a in item['ground_truth']]}")
    print()


## Individual Testing

Let's test each analyzer individually on sample texts:


In [ ]:
test_texts = [
    "The pizza was delicious but the service was terrible.",
    "The laptop has great performance and excellent battery life.",
    "The hotel room was clean and comfortable, but the WiFi was slow."
]

for text in test_texts:
    print(f"\nText: {text}")
    print("-" * 50)
    
    for name, analyzer in analyzers.items():
        try:
            start_time = time.time()
            results = analyzer.analyze(text)
            processing_time = time.time() - start_time
            
            print(f"\n{name} Results ({processing_time:.3f}s):")
            if results:
                for result in results:
                    print(f"  - {result.aspect}: {result.sentiment} (confidence: {result.confidence:.3f})")
            else:
                print("  No aspects found")
                
        except Exception as e:
            print(f"\n{name} Error: {e}")
